# N-Grama

#### Calcular la probabilidad de cada N-Grama.

In [1]:
# Datos
kdata = ["this is a test", "test this thing", "this and that", "I like this"]

In [2]:
#IMPORT SPARK
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [3]:
rsk = sc.parallelize(kdata, 4)

In [4]:
rsk.take(4)

['this is a test', 'test this thing', 'this and that', 'I like this']

In [5]:
def trigrams(t):
    t = t.lower()
    return [t[i:i+3] for i in range(0, len(t) - 2)]

In [6]:
rsk = rsk.flatMap(lambda x:trigrams(x))

In [7]:
rsk.take(21)

['thi',
 'his',
 'is ',
 's i',
 ' is',
 'is ',
 's a',
 ' a ',
 'a t',
 ' te',
 'tes',
 'est',
 'tes',
 'est',
 'st ',
 't t',
 ' th',
 'thi',
 'his',
 'is ',
 's t']

In [8]:
rsk = rsk.map(lambda x:(x,1)).reduceByKey(lambda x,y: x + y)

In [9]:
rsk.take(3)

[('thi', 5), ('s a', 2), (' a ', 1)]

In [10]:
rsk = rsk.sortBy(ascending=False, keyfunc=lambda x: x[1])

In [11]:
rsk.take(5)

[('thi', 5), (' th', 4), ('his', 4), ('is ', 4), ('s a', 2)]

In [15]:
total = rsk.map(lambda x: x[1]).reduce(lambda x,y: x + y)
total

45

In [17]:
rsk = rsk.map(lambda x: (x[0], x[1]/total))

In [19]:
rsk.take(15)

[('thi', 0.1111111111111111),
 (' th', 0.08888888888888889),
 ('his', 0.08888888888888889),
 ('is ', 0.08888888888888889),
 ('s a', 0.044444444444444446),
 ('tes', 0.044444444444444446),
 ('est', 0.044444444444444446),
 (' a ', 0.022222222222222223),
 ('a t', 0.022222222222222223),
 ('hat', 0.022222222222222223),
 ('ike', 0.022222222222222223),
 (' is', 0.022222222222222223),
 (' te', 0.022222222222222223),
 ('st ', 0.022222222222222223),
 ('t t', 0.022222222222222223)]